#### Water Jug Problem

The "water jug" problem can be stated as follows:  
* you have a jug with a capacity of 7 liters of water, and a jug with a capacity of 3 liters of water, 
* you have a water source that will fill either jug to capacity
* you have a drain that can be used to empty either jug  

The available actions are
* fill either jug from the water source
* empty either jug into the drain
* pour some water from a source jug into a destination jug.  The amount actually poured depends on the water level of the source jug and the capacity of the destination jug.   For example
  * if you tried to pour the 3-capacity jug containing 2 liters into the 7-capacity jug containing 6 liters, the 3-capacity jug would contain 1 and the 7-capacity jug would contain 7
  * if you tried to pour the 7-capacity jug containing 3 liters into the 3-capacity jug containing 0 liters, the 3-capacity jug would contain 3 liters and the 7-capacity jug would contain 0 liters
  
It costs 5 to fill either jug regardless of how much water is required.  It costs 3 to empty either jug regardless of how much water is emptied.  It costs 1 to transfer water from one jug to another, regardless of how much water is transferred.

A *problem* consists of 
* Initial -- the initial contents of both jugs
* Goal -- a desired fill level for each jug

So for example, a problem is to find the lowest cost sequence of actions that starts with both jugs empty and results in 0 liters in the 3-liter jug and 3 liters in the 5-liter jug

In [1]:
## First define your world state
##  You should represent your actions as follows:
##     ("fill", "3cap" ) and ("fill", "7cap")
##     ("empty" "3cap") and ("empty", "7cap")
##     ("pour", "3cap", <amount>) and ("pour", "7cap", <amount)  -- where for example
##        ("pour", "3cap")   means "Pour from the 3cap jug to the 7cap jug"
##              The rules above determine how much water is actually poured

from searchClientInterface import WorldState
import copy

class JugWorldState(WorldState):
    """capacities need to be a tuple format, 3 cup first, 7 cup second, same patern for capacities"""
    def __init__(self, max_capacities, initial_volume):
        print('init')
        self.volumes = list(initial_volume)
        self.max_capacities = list(max_capacities)
        self.cost = 0
        print(self.volumes)

    def __str__(self):
        return "{" + str(self.volumes[0]) + "," + str(self.volumes[1]) + "}"
    
    def __eq__(self, other):
        print('eq check')
        if isinstance(other, JugWorldState):
            #return self.value_3 == other.value_3, self.value_7 == other.value_7 
            return self.volumes == other.volumes
        else:
            return False
        
    def __hash__(self):
        print('hash', str(self.volumes))
        return hash(str(self.volumes))
    
    def successors(self):
        candidates = [self.empty(0), self.empty(1),
                      self.pour(0, 1), self.pour(1, 0),
                      self.fill(0), self.fill(1)]
        print('successors check')
        return [c for c in candidates if c] 
    
    
    #Implementing one type of action, where water is transfered
    def pour(self, give, take):
        """cup_give should be the individual object (value_3 or value_7) which is losing water
           cup_take should be the individual object (value_3 or value_7, which is gaining water)
           amount is the quantity of water being transfered"""
        print('before pour', self.volumes, give, take)
        amount = self.volumes[give]
        while int(self.volumes[take] + amount) > self.max_capacities[take]:
            amount -=1
        
        s = copy.deepcopy(self)
        print('amount', amount)
        # add first:
        s.volumes[give] -= amount
        # empty next
        s.volumes[take] += amount
        # is this more complicated than it should be?
        print('pour', s.volumes)
        self.cost += 1
        return((s, "pour"))

    
    #implementing one type of action where water is removed completely
    def empty(self, cup): # cup_empty
        """simple return cup to 0"""
        #if self.volumes != [0,0]: # not necessary, just empty it!
        s = copy.deepcopy(self)
        s.volumes[cup] = 0 # just get rid of all water
        print('empty', s.volumes)
        self.cost += 3
        return((s, "empty"))
        #else:
            #return None
        
    #implementing one type of action where water is added
    def fill(self, cup):
        """simple fill cup up"""
        s = copy.deepcopy(self)
        s.volumes[cup]= self.max_capacities[cup]# need to add max
        self.cost += 5
        print('fill', s.volumes)
        return((s, "fill"))

In [2]:
from searchClientInterface import Problem

class JugProblem(Problem):
    # Create a world state from the capacities and the initials
    # Store off the goal levels for the goal checker
    
    def __init__(self, capacities, initials, goals):
        print('big init')
        self._state = JugWorldState(capacities, initials)
        self._goal_value = goals
    
    # Returns a JugWorldState
    def initial(self):
        print('big initial')
        return self._state
    
    # Checks the input state against the goals to see if they are equal
    def isGoal(self, state):
        print('big isGoal')
        print(state.volumes, '&', list(self._goal_value), state.cost)
        print(state.volumes == list(self._goal_value))
        return state.volumes == list(self._goal_value)

In [3]:
from searchClientInterface import BFSEvaluator, DFSEvaluator
from searchFramework import aStarSearch

# In this cell test your code on a problem where 
#    capacities of the jugs are 3 and 7
#    initial fill levels are both 0
#    goal fill levels are 0 and 5

#  Print the output of both searches
problem1_jug = JugProblem((3,7), (0,0), (0,5))
print(aStarSearch(problem1_jug, BFSEvaluator()))

problem2_jug = JugProblem((3,7), (0,0), (0,5))
print(aStarSearch(problem2_jug, DFSEvaluator()))

#Output Key:(Order of Actions, seconds, cycles, repeats,fringe level)
#Need to improve the way I am handling empty and fill, as it is doing it to both cups, resulting
# in not being able to solve the problem
# need to improve the final print output, as it gives None?

big init
init
[0, 0]
big initial
big isGoal
[0, 0] & [0, 5] 0
False
hash [0, 0]
hash [0, 0]
empty [0, 0]
empty [0, 0]
before pour [0, 0] 0 1
amount 0
pour [0, 0]
before pour [0, 0] 1 0
amount 0
pour [0, 0]
fill [3, 0]
fill [0, 7]
successors check
big isGoal
[0, 0] & [0, 5] 0
False
hash [0, 0]
eq check
big isGoal
[0, 0] & [0, 5] 3
False
hash [0, 0]
eq check
big isGoal
[0, 0] & [0, 5] 6
False
hash [0, 0]
eq check
big isGoal
[0, 0] & [0, 5] 7
False
hash [0, 0]
eq check
big isGoal
[3, 0] & [0, 5] 8
False
hash [3, 0]
hash [3, 0]
empty [0, 0]
empty [3, 0]
before pour [3, 0] 0 1
amount 3
pour [0, 3]
before pour [3, 0] 1 0
amount 0
pour [3, 0]
fill [3, 0]
fill [3, 7]
successors check
big isGoal
[0, 7] & [0, 5] 13
False
hash [0, 7]
hash [0, 7]
empty [0, 7]
empty [0, 0]
before pour [0, 7] 0 1
amount 0
pour [0, 7]
before pour [0, 7] 1 0
amount 3
pour [3, 4]
fill [3, 7]
fill [0, 7]
successors check
big isGoal
[0, 0] & [0, 5] 8
False
hash [0, 0]
eq check
big isGoal
[3, 0] & [0, 5] 11
False
hash [3,

Answer these questions (for this problem only):
* Did DFS return the shortest plan?
* In terms of Nodes explored, which method worked better?

<p style="color:red">Put your answers in this markdown cell</p>

DFS did not return the shortest plan, it iterated much more, as it was much more fixitated on filling to start with

BFS
(['fill', 'pour', 'empty', 'pour', 'empty', 'pour', 'fill', 'pour', 'empty'], (0.078125, 104, 84, 20))
with cost 54

DFS:
(['fill', 'pour', 'fill', 'pour', 'fill', 'pour', 'empty', 'pour', 'empty', 'pour', 'empty', 'pour', 
'fill', 'pour', 'empty'], (0.03125, 51, 35, 41))
with cost 83

The best nodes explored means cheapest score, right, as we see BFS did best, with a cost of only 54 compared to 83.

<p style="color:red">Although I will not run the code your ran to do your analysis, please leave the code in the notebook, in one or more code cells below this cell.</p>

In [ ]:
#BFS
#(['fill', 'pour', 'empty', 'pour', 'empty', 'pour', 'fill', 'pour', 'empty'], (0.078125, 104, 84, 20))
#with cost 54

#DFS:
#(['fill', 'pour', 'fill', 'pour', 'fill', 'pour', 'empty', 'pour', 'empty', 'pour', 'empty', 'pour', 
#'fill', 'pour', 'empty'], (0.03125, 51, 35, 41))
#with cost 83

# The best nodes explored means cheapest score, right? So BFS did the best job?